# プログラム 8-1

In [ ]:
# 認識させる音声をダウンロードする
import urllib.request
url = "https://qrtn.jp/ieu39tu"
filename = "sample.wav"
urllib.request.urlretrieve(url, filename)

In [ ]:
# 音声ファイルを再生する
from IPython.display import Audio
Audio(filename)

In [ ]:
# 音声認識用のライブラリなどをインストールする
!sudo apt update
!sudo apt install ffmpeg
! pip install git+https://github.com/openai/whisper.git -q

In [ ]:
# 準備1 - ライブラリのインポートなど
import whisper
# 音声認識モデルを読み込む
model = whisper.load_model("base")

In [ ]:
# 準備2 - 音声認識に必要なデータを準備する
# 音声データを読み込む
filename = "sample.wav"
audio = whisper.load_audio(filename)
audio = whisper.pad_or_trim(audio)
# 音声データを，音声認識に必要な形式に変換する
mel = whisper.log_mel_spectrogram(audio).to(model.device)
# 認識に必要なオプションを解析する
options = whisper.DecodingOptions()

In [ ]:
# アルゴリズム - 音声認識を実行する
result = whisper.decode(model, mel, options)
result.text # 結果を表示

In [ ]:
# リストのリストで迷路を表現する
maze = [
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
# 迷路のある場所が壁かどうか調べる
x = 0  # 横の位置(左端が0)
y = 1  # 縦の位置(上端が0)
maze[y][x] # 0が道，1が壁

## オマケ

迷路を幅優先権作して，最短経路を調べるPythonのプログラム

In [ ]:
# 上下左右を調べる関数を作る
def find_next(x, y, maze, depth):
    """
    x, yの上下左右を調べ，道として使える場所を返す関数。
    maze(迷路)の中で，調べたい場所をx, yで引数に渡して呼び出す。
    戻り値は，x, yの周囲に道として見つかった位置。
    [[x1, y1]]のように，リストのリストで返す。
    複数の道が見つかったら，[[x1, y1], [x2, y2]]のように要素が増える。
    """
    result_list = []  # 戻り値のリストを空のリストで初期化
    # 上を調べる
    if y > 0 and (maze[y-1][x] == 0 or maze[y-1][x] > depth):
        # 上が道だった
        result_list.append([x, y-1])
    # 下を調べる
    if y < 9 and (maze[y+1][x] == 0 or maze[y+1][x] > depth):
        # 下が道だった
        result_list.append([x, y+1])
    # 左を調べる
    if x > 0 and (maze[y][x-1] == 0 or maze[y][x-1] > depth):
        # 左が道だった
        result_list.append([x-1, y])
    # 右を調べる
    if x < 9 and (maze[y][x+1] == 0 or maze[y][x+1] > depth):
        # 右が道だった
        result_list.append([x+1, y])
    return result_list


In [ ]:
# 迷路を表示する関数を作る
def show_maze(maze):
    """
    リストのリストで表現された迷路を表示する。
    0が道，1が壁。2以上は，探索済みの経路。
    """
    maze_str = "" # 迷路の文字列を初期化
    for line in maze:
        line_str = ""  # 迷路の文字列(一列)を初期化
        for block in line:
            if block == 1:
                line_str += "■"
            elif block == 0:
                line_str += "　"
            else:
                if block >= 2 and block <= 9:
                    line_str += "２３４５６７８９"[block-2]
                else:
                    line_str += str(block)
        maze_str += line_str+"\n"
    print(maze_str)

In [ ]:
# 関数を呼び出してテスト
show_maze(maze)

In [ ]:
# 迷路の探索に使う変数を初期化
stack = []   # 探索の対象となる座標を保存するリストを初期化
route = []   # 答え(最短経路)を保存するリストを初期化

x = 0  # スタート位置(x)
y = 1  # スタート位置(y)
depth = 2  # 探索の深さ

# 迷路を初期化
maze = [
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
# 探索用アルゴリスム(1ターン)
# 結果を確認しながら，何度か実行してみてください

maze[y][x] = depth  # 迷路のリストに，探索深さを記録
route.append([x, y])  # 最短経路のリストに位置を記録
the_way = find_next(x, y, maze, depth)  # 今の位置から移動可能な場所を探す

if len(the_way) > 1:
    # 移動可能な位置が2つ以上だった
    # 2番目以降をスタックに保存しておく
    for item in the_way[1:]:
        # 次の場所のx, 次の場所のy，探索深さ，routeの長さの順に記録
        stack.append([item[0], item[1], depth+1, len(route)])

if len(the_way) == 0:
    # 移動可能な位置がなかった
    # スタックに記録があれば，そこに戻って探索を続ける
    if len(stack) > 0:
        # stackの最後の位置から，座標と探索深さを復元
        item = stack.pop() # stackの一番最後の要素を取り出してから削除
        x = item[0]
        y = item[1]
        depth = item[2]
        route = route[:item[3]]  # ルートを巻き戻す
    else:
        print("ゴールが見つかりました")
else:
    # 移動する
    x = the_way[0][0]
    y = the_way[0][1]
    depth += 1 # 探索深さを+1
    
        
show_maze(maze) # 現在の経過を表示

In [ ]:
# 最短経路を表示(x, yの二重ループを使う)
for y in range(10):
    line_str = ""
    for x in range(10):
        if maze[y][x] == 1:
            line_str += "■"
        else:
            if [x, y] in route:
                line_str += "○"
            else:
                line_str += "　"
    print(line_str)